Pre Processing Data for DTADV-Set

In [59]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from joblib import dump, load
from joblib import dump
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix

In [60]:
%pip install pandas openpyxl

You should consider upgrading via the 'c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [61]:
directory = r'C:\Users\Meyer\Documents\Dom_Detecter\own_data\DTADSV-S\csv'
all_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

In [62]:
list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]
merged_dataframe = pd.concat(list_of_dataframes, ignore_index=True)

In [63]:
df_merged = merged_dataframe

In [64]:
# Read the mapping DataFrame from Excel
mapping_df = pd.read_excel(r'C:\Users\Meyer\Documents\Dom_Detecter\own_data\DTADSV-S\mapping.xlsx', engine='openpyxl')

In [65]:
# Create a mapping dictionary
mapping_dict = dict(zip(mapping_df.src_ip, mapping_df.target))

In [66]:
# Apply the mapping to the 'packet_eth_src' column in df_merged
df_merged['target'] = df_merged['src_ip'].map(mapping_dict)

# Fill any NaN values with 'unknown'
df_merged['target'] = df_merged['target'].fillna('unknown')

In [67]:
flow_counts = df_merged.groupby('target')['Flow'].nunique()

In [68]:
# Count the number of packets for each target and print it
target_counts = df_merged['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown            3603642
tp_link_c200         52627
netatmo_cam          26126
netatmo_weather       1340
Name: count, dtype: int64


In [69]:
df = df_merged

In [70]:
        df.replace('N/A', np.nan, inplace=True)
        df['Start_Time'] = pd.to_datetime(df['Start_Time'])
        df['End_Time'] = pd.to_datetime(df['End_Time'])
        df['Flow_Duration'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds()

        df['Flow_Bytes/s'] = df['Flow_Volume'] / df['Flow_Duration']
        df['Flow_Packets/s'] = df['Packet_Count'] / df['Flow_Duration']

        df.loc[df['Total_Fwd_Packets'] > 0, 'Fwd_IAT'] = df['Inter_Arrival_Time']
        df.loc[df['Total_Bwd_Packets'] > 0, 'Bwd_IAT'] = df['Inter_Arrival_Time']

        feature_df = df.groupby('Flow').agg({
            'Inter_Arrival_Time': ['mean', 'std', 'max', 'min'],
            'Total_Fwd_Packets': ['sum'],
            'Total_Bwd_Packets': ['sum'],
            'Fwd_Packet_Length_Total': ['max', 'min', 'mean', 'std'],
            'Bwd_Packet_Length_Total': ['max', 'min', 'mean', 'std'],
            'Flow_Bytes/s': ['mean'],
            'Flow_Packets/s': ['mean'],
            'packet_length': ['min', 'max', 'mean', 'std', 'var'],
            'Fwd_IAT': ['sum', 'mean', 'std', 'max', 'min'],
            'Bwd_IAT': ['sum', 'mean', 'std', 'max', 'min'],
        })

In [71]:
# Adjust column names and merge
feature_df.columns = ['_'.join(col) for col in feature_df.columns]
feature_df.reset_index(inplace=True)
df = pd.merge(df, feature_df, on='Flow', how='left')

### **DTASV-Set**

In [72]:
clf = load(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\xgboost\xgboost_trained_model.joblib')

In [73]:
df_new = df

In [74]:
# Count the number of packets for each target
target_counts = df['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown            3603642
tp_link_c200         52627
netatmo_cam          26126
netatmo_weather       1340
Name: count, dtype: int64


In [75]:
# Drop duplicate rows based on the 'Flow' column, keeping only the first occurrence
df_new = df_new.drop_duplicates(subset=['Flow'], keep='first')

In [76]:
df_new = pd.get_dummies(df_new, columns=['protocol'])

In [77]:
df_new['ip_id'] = df_new['ip_id'].apply(lambda x: int(x, 16) if isinstance(x, str) else x)

In [78]:
df_new = df_new.fillna(0)

In [79]:
from joblib import dump, load

# Load reference headers
ref_headers = load(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\xgboost\xgboost_header.joblib')

current_headers = df_new.columns

missing_headers = [col for col in ref_headers if col not in current_headers]

for header in missing_headers:
    df_new[header] = 0

df_new = df_new[ref_headers]

In [80]:
X_new = df_new.drop(columns=[
    'target', 'src_ip', 'src_port', 'dst_ip', 'packet_eth_src', 'flow_label',
    'traffic_class', 'Flow', 'Start_Time', 'End_Time', 'next_header', 'ip_len',
    'udp_src_port', 'dsfield_ecn', 'dsfield_dscp', 'flags_mf', 'udp_checksum_status', 'udp_dst_port', 'udp_length','flags_df','ip_hdr_len',
    "packet.sniff_timestamp", "packet_length", "tcp_flags", "app_layer_protocol", "dst_port", "ttl", "Bwd_Packet_Length_Total", "Fwd_Packet_Length_Total",
    "Total_Bwd_Packets", "Total_Fwd_Packets", "ip_id", "Bwd_IAT_min", "packet_length_var", "payload_len", "Fwd_IAT_min", 
    "protocol_TCP", "protocol_UDP", "Flow_Duration", "Fwd_IAT_std", "Flow_Packets/s", "Bwd_Packet_Length_Total_mean"
    ])

In [81]:
Y_new = df_new.target

In [82]:
X_new = X_new.astype(float)

In [83]:
Y_new = Y_new.astype(str)

In [84]:
df_new = df_new.round(8)

In [85]:
X_new.fillna(0, inplace=True)

In [86]:
Y_new_pred = clf.predict(X_new)

c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\xgboost\data.py:359: FutureWarning: is_

In [87]:
Y_new_pred_df = pd.DataFrame(Y_new_pred, columns=['Predicted'])

In [88]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_new_encoded = label_encoder.fit_transform(Y_new.astype(str))

In [89]:
new_accuracy = accuracy_score(Y_new_encoded, Y_new_pred)
print('Accuracy on new data:', new_accuracy)

Accuracy on new data: 0.00900933562947521


In [90]:
all_new_labels = ['amazon_echo', 'belkin_motion', 'belkin_switch', 'netatmo_cam', 'netatmo_weather', 'pixstar_photoframe', 'samsung_cam', 'smart_things', 'tp-link_cam', 'triby_speaker', 'unknown']

In [91]:
cm = confusion_matrix(Y_new, Y_new_pred, labels=all_new_labels)

In [92]:
print("Unique labels in Y_new:", set(Y_new))
print("Unique labels in Y_new_pred:", set(Y_new_pred))

Unique labels in Y_new: {'netatmo_cam', 'netatmo_weather', 'tp_link_c200', 'unknown'}
Unique labels in Y_new_pred: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [93]:
# Original categories
original_labels = ['amazon_echo', 'belkin_motion', 'belkin_switch', 'netatmo_cam', 'netatmo_weather', 'pixstar_photoframe', 'samsung_cam', 'smart_things', 'tp-link_cam', 'triby_speaker', 'unknown']
# Create a label_mapping based on the original categories
label_mapping = {i: label for i, label in enumerate(sorted(original_labels))}

# Convert numerical labels in Y_new_pred to string labels
Y_new_pred_str = [label_mapping[label] for label in Y_new_pred]

# Find all unique labels that appear either in Y_new or Y_new_pred_str
all_labels = sorted(set(Y_new) | set(Y_new_pred_str))

# Generate the confusion matrix using all unique labels
cm = confusion_matrix(Y_new, Y_new_pred_str, labels=all_labels)

# Convert the numpy array to a pandas DataFrame using all unique labels
cm_df = pd.DataFrame(cm, index=all_labels, columns=all_labels)

# Print the DataFrame
print("Confusion Matrix:")
print(cm_df)

Confusion Matrix:
                    amazon_echo  belkin_motion  belkin_switch  netatmo_cam  \
amazon_echo                   0              0              0            0   
belkin_motion                 0              0              0            0   
belkin_switch                 0              0              0            0   
netatmo_cam                   8              6              0            3   
netatmo_weather               2              0              0            0   
pixstar_photoframe            0              0              0            0   
samsung_cam                   0              0              0            0   
smart_things                  0              0              0            0   
tp-link_cam                   0              0              0            0   
tp_link_c200                338              0              1            1   
triby_speaker                 0              0              0            0   
unknown                    1175             78

In [94]:
all_possible_labels = list(set(Y_new) | set(Y_new_pred_str))
label_encoder.fit(all_possible_labels)

LabelEncoder()

In [95]:
# Transform the actual targets
Y_new_encoded = label_encoder.transform(Y_new)
Y_new_pred_encoded = label_encoder.transform(Y_new_pred_str)

In [97]:
cm_df.to_csv(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\xgboost\xgboost_DTADSV-Set\xgboost_DTADVS-Set_cm.csv')

In [98]:
Y_new_df = pd.DataFrame(Y_new, columns=['Actual'])
Y_new_pred_df = pd.DataFrame(Y_new_pred, columns=['Predicted'])

In [99]:
result_df = pd.concat([Y_new_df, Y_new_pred_df], axis=1)

In [100]:
# Create result_df first
Y_new_df = pd.DataFrame(Y_new, columns=['Actual'])
Y_new_pred_df = pd.DataFrame(Y_new_pred, columns=['Predicted'])
result_df = pd.concat([Y_new_df, Y_new_pred_df], axis=1)

In [101]:
Y_test_pred = clf.predict(X_new)

c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\xgboost\data.py:359: FutureWarning: is_

In [102]:
df_new['Predicted_Target'] = Y_new_pred

In [103]:
wrong_predictions = df_new[df_new['target'] != df_new['Predicted_Target']]

In [104]:
wrong_predictions.head()

,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,frame_len,dsfield_dscp,...,Fwd_IAT_max,Fwd_IAT_min,Bwd_IAT_sum,Bwd_IAT_mean,Bwd_IAT_std,Bwd_IAT_max,Bwd_IAT_min,protocol_TCP,protocol_UDP,Predicted_Target
0,fe80::acc:68ff:fe42:391a,0.0,fe80::1e24:cdff:fe81:ac40,0.0,0.0,0,0.0,255.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,False,9
2,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,False,9
3,192.168.1.111,50995.0,192.168.0.23,7680.0,66.0,syn,64240.0,128.0,66.0,0.0,...,8.014908,0.0,0.0,0.0,0.0,0.0,0.0,True,False,9
5,192.168.1.99,0.0,192.168.1.111,0.0,94.0,0,0.0,255.0,94.0,48.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,False,6
6,192.168.1.111,50985.0,51.105.71.136,443.0,60.0,ack,515.0,128.0,60.0,0.0,...,0.000035,0.0,0.0,0.0,0.0,0.0,0.0,True,False,9


In [105]:
print(label_encoder.classes_)


['amazon_echo' 'belkin_motion' 'belkin_switch' 'netatmo_cam'
 'netatmo_weather' 'pixstar_photoframe' 'samsung_cam' 'smart_things'
 'tp-link_cam' 'tp_link_c200' 'triby_speaker' 'unknown']


In [106]:
print(np.unique(Y_new_pred))


[0 1 2 3 4 5 6 7 8 9]


In [107]:
Y_new_pred_original = label_encoder.inverse_transform(Y_new_pred)

In [108]:
# Add these original labels to your result DataFrame
result_df['Predicted_Target_Original'] = Y_new_pred_original

In [109]:
# Also add these to your df_new DataFrame if you want
df_new['Predicted_Target_Original'] = Y_new_pred_original

In [110]:
df_new.head()

,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,frame_len,dsfield_dscp,...,Fwd_IAT_min,Bwd_IAT_sum,Bwd_IAT_mean,Bwd_IAT_std,Bwd_IAT_max,Bwd_IAT_min,protocol_TCP,protocol_UDP,Predicted_Target,Predicted_Target_Original
0,fe80::acc:68ff:fe42:391a,0.0,fe80::1e24:cdff:fe81:ac40,0.0,0.0,0,0.0,255.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,9,tp_link_c200
2,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,9,tp_link_c200
3,192.168.1.111,50995.0,192.168.0.23,7680.0,66.0,syn,64240.0,128.0,66.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,True,False,9,tp_link_c200
5,192.168.1.99,0.0,192.168.1.111,0.0,94.0,0,0.0,255.0,94.0,48.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,6,samsung_cam
6,192.168.1.111,50985.0,51.105.71.136,443.0,60.0,ack,515.0,128.0,60.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,True,False,9,tp_link_c200


In [111]:
df_new['Predicted_Target'] = df_new['Predicted_Target'].map(label_mapping)

In [112]:
df_new.to_csv(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\xgboost\xgboost_DTADSV-Set\xgboost_new_data_DTADSV-Set.csv', index=False)

In [113]:
wrong_predictions.head()

,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,frame_len,dsfield_dscp,...,Fwd_IAT_max,Fwd_IAT_min,Bwd_IAT_sum,Bwd_IAT_mean,Bwd_IAT_std,Bwd_IAT_max,Bwd_IAT_min,protocol_TCP,protocol_UDP,Predicted_Target
0,fe80::acc:68ff:fe42:391a,0.0,fe80::1e24:cdff:fe81:ac40,0.0,0.0,0,0.0,255.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,False,9
2,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,False,9
3,192.168.1.111,50995.0,192.168.0.23,7680.0,66.0,syn,64240.0,128.0,66.0,0.0,...,8.014908,0.0,0.0,0.0,0.0,0.0,0.0,True,False,9
5,192.168.1.99,0.0,192.168.1.111,0.0,94.0,0,0.0,255.0,94.0,48.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,False,6
6,192.168.1.111,50985.0,51.105.71.136,443.0,60.0,ack,515.0,128.0,60.0,0.0,...,0.000035,0.0,0.0,0.0,0.0,0.0,0.0,True,False,9


In [114]:
df_new['is_correct'] = df_new['target'] == df_new['Predicted_Target']

In [115]:
# Group by the true label (i.e., the actual device type)
grouped = df_new.groupby('target')

In [116]:
accuracy_per_device = {}

In [117]:
# Calculate accuracy for each device type
for name, group in grouped:
    accuracy = accuracy_score(group['target'], group['Predicted_Target'])
    accuracy_per_device[name] = accuracy

In [118]:
print("Accuracy per device type:", accuracy_per_device)

Accuracy per device type: {'netatmo_cam': 0.007537688442211055, 'netatmo_weather': 0.047619047619047616, 'tp_link_c200': 0.0, 'unknown': 0.0}
